In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(42)
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from xgboost import XGBRFClassifier, XGBClassifier
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline



In [ ]:
#loading and displaying the data
league_df=pd.read_csv('../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')
display(league_df.head, league_df.info())

In [ ]:
#setting max number of cols displayed to 40
pd.set_option('display.max_columns',40)
league_df.head()

In [ ]:
#dropping game id
league_df.drop('gameId', axis=1, inplace=True)
league_df.head()

In [ ]:
#copying league df to a new df with no removed features
df_feat=league_df.copy()

In [ ]:
#a new df that contains only target variable and controllable parts 
df_league= league_df[['blueWins', 'blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood', 'blueKills','blueDeaths', 'blueAssists', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'redWardsPlaced', 'redWardsDestroyed', 'redFirstBlood', 'redKills','redDeaths', 'redAssists', 'redDragons', 'redHeralds', 'redTowersDestroyed', 'redTotalMinionsKilled', 'redTotalJungleMinionsKilled']]
df_league.head()

In [ ]:
#Function created to visualize correlation and data distribution and outliers
def corr_heatmap(league_df, digits=3, cmap='coolwarm')
"""
created a correlation heatmap to easily visualize multicollinerity that
might be present in the df

Args:
    Dataframe: df with features to check multicollinearity on
    digits(int): number of decimal places to display
    cmap(str):Colormap to display correlation range
Returns:
    fig: Matplotlib Figure
    ax: Matplotlib Axis
"""
#create correlation matrix from df
cor= league_df.corr().round(digits)
cor
#create mask for upper triangle of matrix
mask= np.zeros_like(cor)
mask[np.triu]
